# Interacting with the infra-risk-vis autopackage API

This notebook will:
- Check the remote API is available
- Query the API for available boundaries
- Query the API for available datasets
- Post to the API to trigger an extract for a boundary and set of datasets
- Download files from the server to your local machine for further processing

In [ ]:
import os

import geopandas as gpd

import irv_autopkg_client

In [ ]:
# declare some user inputs up front

# a country to request data for (ISO A3 code)
country_iso = "bgd"

# a location to store requested data
data_dir = "data"

In [ ]:
# create a client object to establish a session
client = irv_autopkg_client.Client()

# is the API responding?
client.server_readiness()

In [ ]:
# which boundaries can we create extracts for?
client.boundary_list()

In [ ]:
# which data processors can we deploy against some boundary?
client.dataset_list()

In [ ]:
# get information on a specific dataset
client.dataset("wri_aqueduct.version_2")

In [ ]:
# submitting a job using one of those data processors
# as an example, all of the WRI Aqueduct flood maps for Egypt
job_id = client.job_submit(
    country_iso,
    [
        "gri_osm.roads_and_rail_version_1",
        "wri_aqueduct.version_2"
    ]
)

In [ ]:
# get the status of the job
client.job_status(job_id)

In [ ]:
# check if the job is complete
client.job_complete(job_id)

In [ ]:
# get the boundary of the territory
boundary = client.boundary_geometry(country_iso)

# save it for later use
os.makedirs(os.path.join(data_dir, country_iso), exist_ok=True)
df = gpd.GeoDataFrame({"name": [country_iso], "geometry": [boundary]})
df.to_file(os.path.join(data_dir, country_iso, "territory.gpkg"))

In [ ]:
# download some extracted data
client.extract_download(
    country_iso,
    data_dir,
    # there may be other datasets available, but only download the following
    dataset_filter=[
        "gri_osm.roads_and_rail_version_1",
        "wri_aqueduct.version_2"
    ],
    overwrite=True
)